In [64]:
import pandas as pd
import nltk
import re

#from itertools import chain
#from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import TfidfModel
from gensim import corpora
from collections import defaultdict
import numpy as np
import math 

from ast import literal_eval
#from textblob import TextBlob
#import spacy
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
#stopwords = set(nltk.corpus.stopwords.words('english'))
#newStopWords = {}

#from nltk.stem import SnowballStemmer
#from nltk.stem import PorterStemmer
#from nltk.stem.lancaster import LancasterStemmer

#sball = SnowballStemmer('english')
#spor = PorterStemmer()
#slan = LancasterStemmer()

import time
import Filter as fil

from collections import Counter, OrderedDict
stopCounter = {}

In [2]:
allData = fil.main("", "", "2001-01-01", "2017-12-31", "", "", "")
allData

,business_id,stars,date,text,name,postal_code,ccur
index,,,,,,,
444610,4JNXUYY8wbaaDmk3BPzlWw,4,2015-07-25,Located next to the Eiffel tower and across th...,"""Mon Ami Gabi""",89109,7328
439722,4JNXUYY8wbaaDmk3BPzlWw,5,2009-12-28,Excellent breakfast spot!\r\r\r\n\r\r\r\nWe ha...,"""Mon Ami Gabi""",89109,7328
439720,4JNXUYY8wbaaDmk3BPzlWw,5,2015-09-13,My wife made reservations for a party of 10. W...,"""Mon Ami Gabi""",89109,7328
439719,4JNXUYY8wbaaDmk3BPzlWw,5,2013-09-22,This place has over 3000+ yelp reviews so we d...,"""Mon Ami Gabi""",89109,7328
439718,4JNXUYY8wbaaDmk3BPzlWw,1,2015-01-21,We made a reservation for 845 we show up at 84...,"""Mon Ami Gabi""",89109,7328
...,...,...,...,...,...,...,...
1394334,G-5kEa6E6PD5fkBRuA7k9Q,4,2015-05-27,We were able to go to Giada's for lunch on our...,"""Giada""",89109,2346
1394335,G-5kEa6E6PD5fkBRuA7k9Q,5,2014-08-23,This was an excellent restaurant to celebrate ...,"""Giada""",89109,2346
1394336,G-5kEa6E6PD5fkBRuA7k9Q,5,2014-10-15,Wonderful dinner experience! We tried to get a...,"""Giada""",89109,2346


In [3]:
def main(item, common, top, granulation, getLemma, frequency, period):
    t0 = time.time()
    if granulation == "Restaurant" or granulation == "restaurant":
        period = ""
    mid = prepareData(item, granulation, getLemma)
    t1 = time.time()
    print("Prepare done: " + str(t1 - t0))
    
    t2 = time.time()
    result = frameY(mid, top, common, frequency, period)
    t3 = time.time()
    print("Frame done: " + str(t3 - t2))
    
    return result

In [4]:
def readCSV(item):
    data = pd.read_csv(item, dtype={0: int}, index_col=0, encoding='latin-1')
    df = pd.DataFrame(data)
    return df

In [5]:
def filterTime(item, time):
    
    result = item.copy()
    indexNames = item[item['time'] != time].index
    result.drop(indexNames , inplace=True)
    result = result.rename(columns={'text': str(time)})
    
    return result[str(time)]

In [6]:
def filterPast(item, time):
    
    result = item.copy()
    indexNames = item[pd.to_datetime(item['time']) > pd.to_datetime(time)].index
    result.drop(indexNames , inplace=True)
    result = result.rename(columns={'text': str(time)})
    
    return result[str(time)]

In [7]:
def prepareData(item, granulation, wordsType):
    
    if (granulation == "Restaurant" or granulation == "restaurant"):
        item["newID"]  = item["name"].map(str) +" - "+ item["postal_code"].map(str)
        item = item.loc[:, ['newID']]
    else:
        item = item.loc[:, ['date']]    
        item = item.sort_values('date', ascending = True)
        
    if granulation == "Year":
        item['time'] = pd.to_datetime(item.date).dt.year
    elif (granulation == "Month"):
        item['time'] = pd.to_datetime(item.date)
        item['time'] = item.time.map(lambda x: x.strftime('%Y-%m'))
        
    elif (granulation == "Restaurant"):
        item['time'] = item.newID# + item.name.postal_code
        
    result = item.loc[:, ['time']]  
    
    t2 = time.time()
    wordsDF = readCSV('static/revSamTok.csv')
    t3 = time.time()
    print("Read done: " + str(t3 - t2))    
    
    result = pd.merge(result, wordsDF[wordsType], left_index=True, right_index=True, how='left')
    result.rename(columns={wordsType: 'text'}, inplace=True)
    
    return result

In [8]:
def frameY(item, top, common, frequency, period):
    start = item.rename(columns={'text': "Overall"})
    
    if frequency == "True" or frequency == "true":
        start = td_idf(start["Overall"], top)
    else: 
        start = tokenCount(start["Overall"], top, common)
    
    uniqueTime = item.time.unique() 
    
    if period == "Past" or period == "past":
        DF_list = [filterPast(item, i) for i in uniqueTime]
    else:
        DF_list = [filterTime(item, i) for i in uniqueTime]
    
    if frequency == "True" or frequency == "true":
        mid = pd.concat([td_idf(i, top) for i in DF_list], axis=1)
    else:    
        mid = pd.concat([tokenCount(i, top, common) for i in DF_list], axis=1)
    
    result = pd.concat([start,mid], axis=1)
    
    result.index+=1
    result.index.names = ['Place']

    return result

In [9]:
def tokenCount(item, top, common):
    global stopCounter
    start = time.time()
    
    allWords = ''.join(item)
    allWords = allWords.replace("][", ", ").strip("']['").split("', '")
    
    t0 = time.time()
    print("Strings done: " + str(t0 - start))
    
    t1 = time.time()
    print("Dist done: " + str(t1 - t0))
    
    allWordDist = Counter(allWords)
    if item.name != "Overall" and common == "false":
        for word in list(allWordDist):
            if word in stopCounter:
                del allWordDist[word]
    
    t2 = time.time()
    print("Counter check: " + str(t2 - t1))
    
    result = tableResults(allWordDist, item.name, top)
    
    if item.name == "Overall" and common == "false":
        stopCounter = result['Words: ' + str(item.name)].tolist()
    
    end = time.time()
    print("Count done: " + str(end - start))
    
    return result

In [336]:
def td_idf(item, top):
    
    t0 = time.time()
    vectorizer = TfidfVectorizer(lowercase=False)
    
    matrix = vectorizer.fit_transform(item).todense()
    #matrix = vectorizer.transform([item])#.todense()
    matrix = pd.DataFrame(matrix, columns=vectorizer.get_feature_names())
    top_words = matrix.sum(axis=0).sort_values(ascending=False)
    
    result = tableResults(top_words, item.name, top)
    
    t1 = time.time()
    print("td_idf - " + item.name + " :" + str(t1-t0))
    
    return result

In [11]:
def tableResults(allWordDist, name, top):
    
    wordCount = pd.Series(allWordDist).to_frame(name)
    wordCount.sort_values(by=[name], inplace = True, ascending=False)
    if (top == "") or (int(top)> wordCount.size):
        top = wordCount.size
    result = wordCount.head(int(top)).reset_index()
    result = result.rename(columns={result.columns[0]: 'Words: ' + str(name)})
    result = result.rename(columns={result.columns[1]: 'Count: ' + str(name)})
    
    return result

In [137]:
start = time.time()
resultat = main(allData.sort_values('date', ascending = True).head(4), "false", "20", "Restaurant", "lemmas", "true", "")
end = time.time()
print("All done: " + str(end - start))
resultat

Read done: 1.876986026763916
Prepare done: 1.9268321990966797
td_idf - Overall :0.005979061126708984
td_idf - "Burger Bar" - 89109 :0.004986763000488281
td_idf - "Mesa Grill" - 89109 :0.003990888595581055
td_idf - "Lotus of Siam" - 89104 :0.003992319107055664
Frame done: 0.029920101165771484
All done: 2.0585031509399414


,Words: Overall,Count: Overall,"Words: ""Burger Bar"" - 89109","Count: ""Burger Bar"" - 89109","Words: ""Mesa Grill"" - 89109","Count: ""Mesa Grill"" - 89109","Words: ""Lotus of Siam"" - 89104","Count: ""Lotus of Siam"" - 89104"
Place,,,,,,,,
1,burger,0.581493,burger,0.525757,food,0.324443,thai,0.337526
2,food,0.458839,concept,0.283846,restaruant,0.162221,food,0.225018
3,thai,0.358485,quality,0.265856,spectacular,0.162221,thing,0.225018
4,taste,0.293421,get,0.265856,spanish,0.162221,taste,0.225018
5,good,0.284296,cheddar,0.265856,simply,0.162221,restaurant,0.225018
6,concept,0.282718,good,0.256478,side,0.162221,world,0.225018
7,quality,0.266355,little,0.189231,server,0.162221,typical,0.112509
8,cheddar,0.266355,bar,0.189231,select,0.162221,suit,0.112509
9,get,0.266355,five,0.189231,lunch,0.162221,thailand,0.112509


In [148]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(proba4["text"])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,2nd,30,40,afternoon,appetizer,area,ask,back,bag,bagel,...,toast,tourist,trade,ufeff1,underwhelm,us,vega,wait,watch,would
0,0.000000,0.000000,0.139363,0.000000,0.139363,0.000000,0.000000,0.000000,0.000000,0.139363,...,0.139363,0.139363,0.139363,0.000000,0.139363,0.000000,0.000000,0.000000,0.000000,0.139363
1,0.000000,0.134599,0.000000,0.134599,0.000000,0.134599,0.134599,0.134599,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.134599,0.000000,0.134599,0.000000,0.000000
2,0.197969,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.197969,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.197969,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.265962,0.000000,0.000000,0.265962,0.000000,0.000000,0.000000


In [116]:
def dontTokenise(item):
    allWords = list(item.strip("']['").split("', '"))
    return allWords

In [366]:
def gensimTDIDF(item, top):
    t0 = time.time()
    
    #a = list(i for i in item)
    #for i in range(len(a)):
    #    a[i] = dontTokenise(a[i])

    #dataset = a
    dataset = item
    
    
    t1 = time.time()
    print("Pre done :" + str(t1-t0))
    
    dictionary = corpora.Dictionary(dataset)
    corpus = [dictionary.doc2bow(word) for word in dataset]  # convert corpus to BoW format
    model = TfidfModel(corpus,smartirs="dnn")
    #print(str(model))
    corpus_tfidf  = model[corpus]
    #print(str(corpus_tfidf[1]))
    
    t7 = time.time()
    print("Model done :" + str(t7-t1))

    #d = {dictionary.get(id): value for doc in corpus_tfidf for id, value in doc}
   # for doc in corpus_tfidf:
    #    d = [[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc]
    
    #d = corpus_tfidf[1]
    #d = [(dictionary[pair[0]], pair[1]) for pair in d]
    #print(d)
    
    d = {}
    for doc in corpus_tfidf:
        print(doc)
        for id, value in doc:
            word = dictionary.get(id)
            print(str(word) +" "+ str(value))
            #print(value)
            d[word] = value
    
    print(d)
    
    t8 = time.time()
    print("D done :" + str(t8-t7))
    
    nanana = tableResults(d, item.name, int(top))      
    
    t9 = time.time()
    print("tableResults done :" + str(t9-t8))
    print("All done :" + str(t9-t0))
    
    return nanana

In [367]:
gensimTDIDF(proba1["text"].head(3), "10")
#nananaSeries = pd.Series([["gooood", "jedi","you"], ["gooood", "gooood","anakin","gooood"], ["anakin", "anakin"]],[1, 2,3])
#nananaSeries.name = "text"
#gensimTDIDF(nananaSeries, "10")
#nananaSeries

Pre done :0.0
Model done :0.0
[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0), (15, 1.0), (16, 1.0)]
affordable 1.0
bellagio 1.0
classic 1.0
fairly 1.0
food 1.0
fountain 1.0
french 1.0
like 1.0
outside 1.0
paris 1.0
people 1.0
reminiscent 1.0
see 1.0
show 1.0
sit 1.0
steroids 1.0
watch 1.0
[(4, 1.0), (17, 2.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 1.0), (22, 1.0), (23, 1.0), (24, 1.0), (25, 1.0), (26, 1.0), (27, 1.0), (28, 1.0), (29, 1.0), (30, 1.0), (31, 1.0), (32, 1.0), (33, 1.0), (34, 1.0), (35, 2.0), (36, 1.0), (37, 1.0), (38, 1.0), (39, 1.0), (40, 1.0), (41, 2.0)]
food 1.0
4 2.0
5 1.0
admit 1.0
although 1.0
amazing 1.0
best 1.0
bottle 1.0
call 1.0
direct 1.0
dumb 1.0
glasses 1.0
going 1.0
high 1.0
make 1.0
meal 1.0
must 1.0
numerous 1.0
ordering 1.0
price 2.0
rather 1.0
result 1.0
star 1.0
super 1.0
vegas 1.0
wine 2.0
[(7, 1.0), (9, 1.0), (18, 1.0), (35, 1.0), (40, 1.0), (41

,Words: text,Count: text
0,definitely,2.370143
1,mon,2.000000
2,beef,2.000000
3,gabi,2.000000
4,wine,2.000000
5,ami,2.000000
6,experience,2.000000
7,one,2.000000
8,made,2.000000
9,4,2.000000


In [349]:
len(proba1["text"].head(3).iloc[2])

83

In [335]:
td_idf(proba1["text"].head(2), "10")

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.

In [13]:
def tokenisedList(item):
    result = list(item.strip("']['").split("', '"))
    return result

In [103]:
def computeTF(item):
    #allWords = tokenisedList(item)
    allWords = item
    allWords = Counter(allWords)
    n = sum(allWords.values())
    
    result = Counter({k:v/n for k,v in allWords.items()})
    return result

In [210]:
def computeIDF(item, word):
    N = len(item)
    DFt = 0
    for x in range(len(item)):
        #tokenList = tokenisedList(item.iloc[x])
        #tokenList = item.iloc[x]
        if word in item.iloc[x]:
            DFt = DFt + 1
    result = math.log(N/DFt)
    return result

In [214]:
def computeTF_IDF(item, i, ss):
    t0 = time.time()
    tf = computeTF(item.iloc[i])
    t1 = time.time()    
    #print("computeTF of " + str(i) + " : " + str(t1-t0))
    for word in tf:
        t1_1 = time.time()        
        idf = computeIDF(ss, word)
        tf[word] = tf[word] * idf
        t2_2 = time.time()
        #print("CycleIDF of " + str(word) + " : " + str(t2_2-t1_1))
    t2 = time.time()
    #print("computeIDF of " + str(i) + " : " + str(t2-t1))
    
    return tf

In [217]:
def fullTF_IDF(item):
    c = Counter()
    #SetsOfSeries = item.apply(set)
    SetsOfSeries = item
    #print(SetsOfSeries)
    for i in range(len(item)):
        t0 = time.time()
        x = computeTF_IDF(item, i, SetsOfSeries)
        t1 = time.time()
        #print("fullTF_IDF of " + str(i) + " : " + str(t1-t0))
        c = c + x
        #print(c)
    return c

In [221]:
t0 = time.time()
nakraq = fullTF_IDF(proba1.head(60)["text"])
t1 = time.time()
print("All done : " + str(t1-t0))
#nakraq

All done : 2.179177761077881


In [102]:
t0 = time.time()
nakraq = fullTF_IDF(proba0.head(10)["text"])
t1 = time.time()
print("All done : " + str(t1-t0))
nakraq

All done : 0.11466598510742188


Counter({'affordable': 0.1304381053214565,
         'fairly': 0.13544618194082622,
         'classic': 0.13544618194082622,
         'french': 0.09456661552239931,
         'food': 0.12753746743265504,
         'sit': 0.11442052282552068,
         'outside': 0.10301366878829916,
         'like': 0.11406363660057409,
         'people': 0.13544618194082622,
         'watch': 0.11799800551520259,
         'see': 0.106251508395988,
         'bellagio': 0.057564840500127466,
         'fountain': 0.08686994074035577,
         'show': 0.08411267743106576,
         'reminiscent': 0.13544618194082622,
         'paris': 0.0708403663561401,
         'steroids': 0.13544618194082622,
         'wine': 0.1268166725159329,
         'amazing': 0.06537171127297382,
         'super': 0.07939948596531193,
         'high': 0.07939948596531193,
         'price': 0.11322372701437412,
         'going': 0.07939948596531193,
         'make': 0.06504184023051347,
         '4': 0.12257440811641497,
         'star

In [101]:
mid = allData.head(4000)
mid
proba1 = prepareData(mid, "Year", "tokens")
proba1["text"] = proba1["text"].apply(literal_eval)
proba1
proba0 = prepareData(mid, "Year", "tokens")
proba0

Read done: 1.9916739463806152
Read done: 1.950789213180542


,time,text
index,,
440290,2005,"['affordable', 'fairly', 'classic', 'french', ..."
441054,2006,"['food', 'wine', 'amazing', 'super', 'high', '..."
444703,2006,"['went', 'team', 'function', 'one', 'evening',..."
439162,2006,"['best', 'friend', 'took', 'good', 'experience..."
442019,2006,"['affordable', 'dining', 'experience', 'paris'..."
...,...,...
440285,2017,"['usually', 'get', 'right', 'time', 'ordered',..."
440251,2017,"['try', 'get', 'seat', 'patio', 'weather', 'ni..."
438956,2017,"['outstanding', 'overall', 'dining', 'experien..."


In [148]:
proba1.head(2)["text"]

index
440290    [affordable, fairly, classic, french, food, si...
441054    [food, wine, amazing, super, high, price, goin...
Name: text, dtype: object

In [228]:
t0=time.time()
#listche = ["good","good","good", "anakin", "good"]
proba2 = proba1.head(4)["text"].apply(set)
#proba2 = proba1.head(1000)["text"]#.apply(set)
#listche = proba1["text"].iloc[0]
DFt = 0
for row in range(len(proba2)):
    if "food" in proba2.iloc[row]:
        DFt = DFt + 1
DFt
t1=time.time()
print("All done : " + str(t1-t0))
DFt
#proba2

All done : 0.0020241737365722656


3